In [4]:
!pip3 install -r requirements.txt

  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.5 MB/s eta 0:00:00
Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
Using cached smmap-5.0.1-py3-none-any.whl (24 kB)


In [1]:
from dotenv import dotenv_values
import os
SECRETS=dotenv_values(".env")
os.environ['OPENAI_API_KEY'] = SECRETS['OPENAI_API_KEY']

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAI

embeddings = OpenAIEmbeddings(show_progress_bar=True)

llm = OpenAI(temperature=0)

/Users/rayanaay/anaconda3/envs/langchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


# Initializing GitLoader : Polars

In [14]:
from langchain.document_loaders import GitLoader

loader = GitLoader(
    clone_url="https://github.com/pola-rs/polars",
    repo_path="./data/repo/",
    file_filter=lambda file_path: file_path.endswith(".rs"),
    branch="main",
)

documents = loader.load()

In [17]:
print(documents[0])

page_content='/// Build script using \'built\' crate to generate build info.\n\nfn main() {\n    println!("cargo::rustc-check-cfg=cfg(allocator, values(\\"default\\", \\"mimalloc\\"))");\n\n    #[cfg(feature = "build_info")]\n    {\n        println!("cargo:rerun-if-changed=build.rs");\n        extern crate built;\n        use std::env;\n        use std::path::Path;\n\n        // We must specify the workspace root as the source\n        let py_polars_dir = env::var("CARGO_MANIFEST_DIR").unwrap();\n        let src = Path::new(&py_polars_dir).parent().unwrap();\n\n        let out_dir = &env::var("OUT_DIR").unwrap();\n        let dst = Path::new(&out_dir).join("built.rs");\n\n        built::write_built_file_with_opts(Some(src), &dst)\n            .expect("Failed to acquire build-time information");\n    }\n}\n' metadata={'source': 'py-polars/build.rs', 'file_path': 'py-polars/build.rs', 'file_name': 'build.rs', 'file_type': '.rs'}


In [22]:
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter

# so that LLM can more easily understand the structure

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.RUST, 
    chunk_size=1000, 
    chunk_overlap=200
)

documents = python_splitter.split_documents(documents)

In [24]:
print(documents[0])

page_content='/// Build script using \'built\' crate to generate build info.\n\nfn main() {\n    println!("cargo::rustc-check-cfg=cfg(allocator, values(\\"default\\", \\"mimalloc\\"))");\n\n    #[cfg(feature = "build_info")]\n    {\n        println!("cargo:rerun-if-changed=build.rs");\n        extern crate built;\n        use std::env;\n        use std::path::Path;\n\n        // We must specify the workspace root as the source\n        let py_polars_dir = env::var("CARGO_MANIFEST_DIR").unwrap();\n        let src = Path::new(&py_polars_dir).parent().unwrap();\n\n        let out_dir = &env::var("OUT_DIR").unwrap();\n        let dst = Path::new(&out_dir).join("built.rs");\n\n        built::write_built_file_with_opts(Some(src), &dst)\n            .expect("Failed to acquire build-time information");\n    }\n}' metadata={'source': 'py-polars/build.rs', 'file_path': 'py-polars/build.rs', 'file_name': 'build.rs', 'file_type': '.rs'}


# Create chain to augment the LLM

In [26]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler
from langchain.vectorstores import FAISS


llm = ChatOpenAI()

index = FAISS.from_documents(documents, embeddings)
retriever = index.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,
)
handler = StdOutCallbackHandler()

query = "What is a dataframe?"

qa.run(query,
      callbacks=[handler])

100%|██████████| 15/15 [01:44<00:00,  7.00s/it]




> Entering new RetrievalQA chain...


100%|██████████| 1/1 [00:00<00:00,  1.86it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
fn load_df() -> DataFrame {
    df!("a" => &[1, 2, 3, 4, 5],
                 "b" => &["a", "a", "b", "c", "c"],
                 "c" => &[1, 2, 3, 4, 5]
    )
    .unwrap()
}

use std::io::Cursor;

use optimization_checks::*;
use polars_core::chunked_array::builder::get_list_builder;
use polars_core::df;
#[cfg(feature = "temporal")]
use polars_core::export::chrono::{NaiveDate, NaiveDateTime, NaiveTime};
use polars_core::prelude::*;
#[cfg(feature = "parquet")]
pub(crate) use polars_core::SINGLE_LOCK;
use polars_io::prelude::*;

#[cfg(feature = "cov")]
use crate::dsl::pearson_corr;
use crate::prelude::*;

accumulate_dataframes_horizontal(cols)
    }
}

///
/// assert_eq!(df[0], Series::new("Fruit", &[

'A DataFrame is a tabular data structure in Polars, which is a Rust crate for data manipulation and analysis. It consists of columns and rows, where each column can have different data types. It is used for handling and analyzing structured data efficiently.'

In [27]:
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs['fetch_k'] = 20
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 20

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,
)

query = "When should I use Avro ?"

qa.run(query, callbacks=[handler])



> Entering new RetrievalQA chain...


100%|██████████| 1/1 [00:01<00:00,  1.58s/it]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
AvroSchema::Null,
                    AvroSchema::Record(Record {
                        name: "r2".to_string(),
                        namespace: None,
                        doc: None,
                        aliases: vec![],
                        fields: vec![
                            AvroField {
                                name: "item1".to_string(),
                                doc: None,
                                schema: AvroSchema::Int(None),
                                default: None,
                                order: None,
                                aliases: vec![],
                            },
                            AvroField {
                       

"Avro can be used when you need a data serialization framework that provides features such as schema evolution, compact binary encoding, and efficient serialization and deserialization. It is commonly used in distributed systems, data storage, and data processing applications where data interchange between different systems or components is required. Avro's support for schema evolution makes it a good choice for scenarios where data schemas may evolve over time without breaking compatibility."